In [31]:
with open("input.txt") as f:
    for line in f:
        print(line.strip())

MMMMMMMMMMMMWWWWWWWWAAAAAAAAAAAAAAAASSSSSSHHHIIHHHHHHHGGGGGGGGGXXXXXXXXEEEEEEENPMMMMMMMMMMPPPPPPPPPPPPPPPAAAHHPHHHHHHHHHHHHTTTTTTTTBBBBWWRRR
MMMMMMMMMMWWWWWWWWWAAAAAAAAAAAAAAAAASSSSSSHHHHHHHHHHGHGGGGGGGGGXXXXXXEEEEEEEENNNMMMMMMMMMMPPPPPPPPPPPPPPPAAAHHHHHHHHHHHHHHHDTTTTQTBBBBWWWWRR
MMMMMMMMMMMWWWWWWWWAAAAAAAAAAAAAAAAAASSSHHHHHHHHHHHHGGGGGGGGGGGXXXXXEEEEEENNNNNMMMMMMMMMMFPPPPPPPPPPPPPPPAAAHHHHHHHHHHHHHHHHTTTLTBBBBWWWWWWW
MMMMMMMMMMMWVWWWWAAAAAAAAAAAAAAAAAAASSSSHHHHHHHHHHTTGGGGGGGGGGXXXXXXEEEEEEENNNNNMMMMMMMMMPPPPPPPPPPPPPPPPPAAHHHHHHHHHHHHHHHHTTTTTBBBBBBWWWWW
MMMMMMMMMMWWWWWWWWAAAAAAAAAAAAAAAAAAAASSHHHHHHHHHHTHGGGGGGGGGGXXXXEEEEEEEEENNNNNMMMMMMMMMPPPPPPPPPPPPPPPPPRRHHHHHHHHHHHHHHHHTTTTBBBBBWWWWWWW
MMMMMMMMMWWWWWWWWWAAAAAAAAAAAAAAAAAAAAUSHHHHHHHHHHHHHGGGGGGGGGGXXXXEESEEEENNNNNMMMMMMMMPPPPPPPPPPPPPPPPPPPRRRHDDHHHHHHHHHHHHTTTTTTBBWWWWWWWW
MMMMMMMMMXWWWWWWWAAAAAAAAAAAAAAAAAAAAESSJHHHHHHHHHHHHGGGGGGGGGGGXXEEESEEEEEENNMMMMMMMMMPPPPPPPPPPPPPPPPGGPRRRRRRRHHHHHHHHHHRRRRTHHHBWWWWWWWW
MMMMMMMMMWWWW

In [71]:
from enum import Enum

class Coord:
    def __init__(self, x: int, y: int, value: str):
        self.x = x
        self.y = y
        self._value = value

    def is_linked_to(self, other: "Coord") -> bool:
        return abs(self.x - other.x) + abs(self.y - other.y) == 1 and (other.value == self.value)

    @property
    def value(self):
        return self._value
    
    def is_north(self, other: "Coord") -> bool:
        return self.is_linked_to(other) and self.y > other.y
    
    def is_south(self, other: "Coord") -> bool:
        return self.is_linked_to(other) and self.y < other.y
    
    def is_east(self, other: "Coord") -> bool:
        return self.is_linked_to(other) and self.x < other.x
    
    def is_west(self, other: "Coord") -> bool:
        return self.is_linked_to(other) and self.x > other.x
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f"({self.x},{self.y}) {self.value}"
    
class Fence(Enum):
    NORTH = 1
    EAST = 2
    SOUTH = 3
    WEST = 4



class CoordFence(Coord):
    def __init__(self, x: int, y: int, value: str, fences: list[Fence]):
        super().__init__(x, y, value)
        self.fences = fences
    
    def __repr__(self):
        return f"({self.x},{self.y}) {self.value} <{','.join([str(f) for f in self.fences])}>"
    


    

class Graph:
    def __init__(self, lines: list[list[str]]):
        self.lines = lines
        self.width = len(lines[0])
        self.height = len(lines)
        self.patches: list[list[Coord]] = []
        self.patches_with_fences: list[list[CoordFence]] = []
        self.bags: dict[Coord, list[Coord]] = {}


        for y, line in enumerate(lines):
            for x, value in enumerate(line):
                coord = Coord(x, y, value)

                if x > 0 and coord.is_linked_to(self.coord(x-1, y)):
                    self.add_edge(coord, self.coord(x-1, y))
                if x < self.width - 1 and coord.is_linked_to(self.coord(x+1, y)):
                    self.add_edge(coord, self.coord(x+1, y))
                if y > 0 and coord.is_linked_to(self.coord(x, y-1)):
                    self.add_edge(coord, self.coord(x, y-1))
                if y < self.height - 1 and coord.is_linked_to(self.coord(x, y+1)):
                    self.add_edge(coord, self.coord(x, y+1))

        self.calc_patches()
        self.calc_patches_with_fences()

    def coord(self, x: int, y: int) -> Coord:
        return Coord(x, y, self.lines[y][x])

    def add_edge(self, a: Coord, b: Coord):
        if a not in self.bags:
            self.bags[a] = [b]
        else:
            self.bags[a].append(b)

    def neighbors(self, x: int, y: int) -> list[Coord]:
        return self.bags.get(Coord(x, y, ""), [])
    

    def _fences(self, c: Coord) -> list[Fence]:
        neighbors = self.neighbors(c.x, c.y)
        fences: list[Fence] = []
        has_north = False
        has_east = False
        has_south = False
        has_west = False
        for n in neighbors:
            if c.is_north(n):
                has_north = True
            if c.is_east(n):
                has_east = True
            if c.is_south(n):
                has_south = True
            if c.is_west(n):
                has_west = True
        if not has_north:
            fences.append(Fence.NORTH)
        if not has_east:
            fences.append(Fence.EAST)
        if not has_south:
            fences.append(Fence.SOUTH)
        if not has_west:
            fences.append(Fence.WEST)

        return fences
    
    def calc_patches_with_fences(self):
        for patch in self.patches:
            new_patch = []
            for coord in patch:
                coord_fence = CoordFence(coord.x, coord.y, coord.value, self._fences(coord))
                new_patch.append(coord_fence)
            self.patches_with_fences.append(new_patch)
    
    def calc_patches(self):
        visited = set()
        for y in range(self.height):
            for x in range(self.width):
                coord = self.coord(x, y)
                if coord not in visited:
                    patch = self._dfs(coord, visited)
                    self.patches.append(patch)
    
    def _dfs(self, coord: Coord, visited: set[Coord]) -> list[Coord]:
        visited.add(coord)
        result = [coord]
        for neighbor in self.neighbors(coord.x, coord.y):
            if neighbor not in visited:
                result += self._dfs(neighbor, visited)
        return result
    
    def __repr__(self):
        return "\n".join("".join(str(i) for i in line) for line in self.lines)
    

In [72]:
graph = Graph([list(line.strip()) for line in open("example.txt")])
graph

RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE

In [53]:
visited: set[Coord] = set()
all_visited: set[Coord] = set()

def calc_circumference(patch: list[Coord]) -> int:
    sum = 0
    for c in patch:
        n_neighbours = graph.neighbors(c.x, c.y)
        sum += 4 - len(n_neighbours)
    return sum


total = 0
for p in graph.patches:
    circ = calc_circumference(p)
    # print(circ, len(p))
    
    total += circ * len(p)
print(total)


1930


# Part 2

In [56]:
graph

RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE

In [74]:
graph.patches_with_fences

[[(0,0) R <Fence.NORTH,Fence.WEST>,
  (1,0) R <Fence.NORTH>,
  (2,0) R <Fence.NORTH>,
  (3,0) R <Fence.NORTH,Fence.EAST>,
  (3,1) R <Fence.EAST>,
  (2,1) R <>,
  (1,1) R <Fence.SOUTH>,
  (0,1) R <Fence.SOUTH,Fence.WEST>,
  (2,2) R <Fence.WEST>,
  (3,2) R <Fence.SOUTH>,
  (4,2) R <Fence.NORTH,Fence.EAST,Fence.SOUTH>,
  (2,3) R <Fence.EAST,Fence.SOUTH,Fence.WEST>],
 [(4,0) I <Fence.NORTH,Fence.WEST>,
  (5,0) I <Fence.NORTH,Fence.EAST>,
  (5,1) I <Fence.EAST,Fence.SOUTH>,
  (4,1) I <Fence.SOUTH,Fence.WEST>],
 [(6,0) C <Fence.NORTH,Fence.WEST>,
  (7,0) C <Fence.NORTH,Fence.EAST>,
  (7,1) C <Fence.SOUTH>,
  (6,1) C <Fence.WEST>,
  (6,2) C <Fence.EAST,Fence.SOUTH>,
  (5,2) C <Fence.NORTH,Fence.WEST>,
  (5,3) C <Fence.EAST,Fence.SOUTH>,
  (4,3) C <Fence.NORTH>,
  (3,3) C <Fence.NORTH,Fence.SOUTH,Fence.WEST>,
  (4,4) C <Fence.EAST,Fence.WEST>,
  (4,5) C <Fence.SOUTH,Fence.WEST>,
  (5,5) C <Fence.NORTH,Fence.EAST>,
  (5,6) C <Fence.EAST,Fence.SOUTH,Fence.WEST>,
  (8,1) C <Fence.NORTH,Fence.EAST

In [82]:
def min_max(patch: list[CoordFence]) -> tuple[int, int, int, int]:
    min_x = min([c.x for c in patch])
    max_x = max([c.x for c in patch])
    min_y = min([c.y for c in patch])
    max_y = max([c.y for c in patch])
    return min_x, max_x, min_y, max_y

def get_coord(patch: list[CoordFence], x: int, y: int) -> CoordFence:
    for c in patch:
        if c.x == x and c.y == y:
            return c
    return None

def calc_fence(patch: list[CoordFence]) -> int:
    min_x, max_x, min_y, max_y = min_max(patch)
    
    sides_total = 0
    for x in range(min_x, max_x+1):
        sides_east = 0
        sides_west = 0
        track_east = False
        track_west = False
        for y in range(min_y, max_y+1):
            coord_fence = get_coord(patch, x, y)
            if coord_fence is not None and Fence.EAST in coord_fence.fences:
                if not track_east:
                    sides_east += 1
                track_east = True
            else:
                track_east = False
            
            if coord_fence is not None and Fence.WEST in coord_fence.fences:
                if not track_west:
                    sides_east += 1
                track_west = True
            else:
                track_west = False
        sides_total += sides_east + sides_west

    for y in range(min_y, max_y+1):
        sides_north = 0
        sides_south = 0
        track_north = False
        track_south = False
        for x in range(min_x, max_x+1):
            coord_fence = get_coord(patch, x, y)
            if coord_fence is not None and Fence.NORTH in coord_fence.fences:
                if not track_north:
                    sides_north += 1
                track_north = True
            else:
                track_north = False
            
            if coord_fence is not None and Fence.SOUTH in coord_fence.fences:
                if not track_south:
                    sides_south += 1
                track_south = True
            else:
                track_south = False
        sides_total += sides_north + sides_south

    return sides_total * len(patch)


In [90]:
print(graph.patches_with_fences[0])

calc_fence(graph.patches_with_fences[0])

[(0,0) R <Fence.NORTH,Fence.WEST>, (1,0) R <Fence.NORTH>, (2,0) R <Fence.NORTH>, (3,0) R <Fence.NORTH,Fence.EAST>, (3,1) R <Fence.EAST>, (2,1) R <>, (1,1) R <Fence.SOUTH>, (0,1) R <Fence.SOUTH,Fence.WEST>, (2,2) R <Fence.WEST>, (3,2) R <Fence.SOUTH>, (4,2) R <Fence.NORTH,Fence.EAST,Fence.SOUTH>, (2,3) R <Fence.EAST,Fence.SOUTH,Fence.WEST>]


120

In [93]:
graph = Graph([list(line.strip()) for line in open("input.txt")])

total = 0
for p in graph.patches_with_fences:
    total += calc_fence(p)
print(total)

923480
